In [102]:
import json
import pandas as pd
import time
import numpy as np
import os, json, glob
from postal.parser import parse_address

In [103]:
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_colwidth', -1)

/Users/maryamnuri/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


#### Residential sales

In [57]:
iproperty = pd.read_csv('')
iproperty.shape

(34469, 13)

In [58]:
iproperty.head()

,state,type,min_price,max_price,address,lat,lng,landArea,builtUp,sizeUnit,level1,level2,level3
0,johor,sale,380000.0,380000,"jalan sri putri 7/x, taman putri, 81000, Johor",1.656731,103.572500,NaN,1540,SQUARE_FEET,Johor,Kulai,NaN
1,johor,sale,518000.0,518000,"jalan kebangsaan, Taman Universiti, Pulai Utama, 81300, Johor",1.533162,103.629529,309848,1300,SQUARE_FEET,Johor,Skudai,NaN
2,johor,sale,1380000.0,1380000,"jalan indah 14/x, Bukit Indah, 79100, Johor",1.483319,103.645116,3200,3367,SQUARE_FEET,Johor,Johor Bahru,NaN
3,johor,sale,313500.0,512260,"Bandar Putera Indah, Batu Pahat, Johor",1.938611,102.933500,26 acres,From 1498 - 1602,NaN,Johor,Batu Pahat,Emerald Garden 3
4,johor,sale,600000.0,600000,"Setia Tropika, Setia Tropika, 81200, Johor",NaN,NaN,1500,2100,SQUARE_FEET,Johor,Setia Tropika,NaN


In [59]:
iproperty.groupby('state')['address'].nunique().reset_index()

,state,address
0,johor,2298
1,kedah,1541
2,kelantan,318
3,kuala-lumpur,1143
4,labuan,1
5,melaka,1350
6,negeri-sembilan,1791
7,pahang,663
8,penang,1270
9,perak,1856


In [60]:
iproperty.state.unique()

array(['johor', 'kedah', 'kelantan', 'melaka', 'negeri-sembilan',
       'pahang', 'penang', 'perak', 'perlis', 'sabah', 'sarawak',
       'selangor', 'terengganu', 'kuala-lumpur', 'labuan', 'putrajaya'],
      dtype=object)

In [61]:
# iproperty['libpostal_address'] = iproperty["address"].apply(parse_address)
# iproperty.head()

In [62]:
all_states = iproperty.drop_duplicates(subset=['address',
                                            'lat','lng',
                                            'min_price','max_price',
                                             'builtUp'], keep='first')

In [63]:
all_states.shape

(30966, 13)

In [64]:
all_states.head()

,state,type,min_price,max_price,address,lat,lng,landArea,builtUp,sizeUnit,level1,level2,level3
0,johor,sale,380000.0,380000,"jalan sri putri 7/x, taman putri, 81000, Johor",1.656731,103.572500,NaN,1540,SQUARE_FEET,Johor,Kulai,NaN
1,johor,sale,518000.0,518000,"jalan kebangsaan, Taman Universiti, Pulai Utama, 81300, Johor",1.533162,103.629529,309848,1300,SQUARE_FEET,Johor,Skudai,NaN
2,johor,sale,1380000.0,1380000,"jalan indah 14/x, Bukit Indah, 79100, Johor",1.483319,103.645116,3200,3367,SQUARE_FEET,Johor,Johor Bahru,NaN
3,johor,sale,313500.0,512260,"Bandar Putera Indah, Batu Pahat, Johor",1.938611,102.933500,26 acres,From 1498 - 1602,NaN,Johor,Batu Pahat,Emerald Garden 3
4,johor,sale,600000.0,600000,"Setia Tropika, Setia Tropika, 81200, Johor",NaN,NaN,1500,2100,SQUARE_FEET,Johor,Setia Tropika,NaN


In [65]:
all_states["price_mean"] = np.where(
    all_states["max_price"] != 0, 
    (all_states["min_price"] + all_states["max_price"])/2, 
    all_states["min_price"]
)

/Users/maryamnuri/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [66]:
# Capped property price at 5mil
all_states = all_states[all_states['price_mean']<=5000000]

In [67]:
all_states.price_mean.max()

5000000.0

In [68]:
# Principal amount estimate
all_states['principal_amt'] = all_states['price_mean']*0.9

In [69]:
# Monthly payment estimate
all_states['monthly_pmt'] = all_states['principal_amt']*((0.045/12)*((1+0.045/12)**360))/(((1+0.045/12)**360)-1)

In [70]:
all_states.head()

,state,type,min_price,max_price,address,lat,lng,landArea,builtUp,sizeUnit,level1,level2,level3,price_mean,principal_amt,monthly_pmt
0,johor,sale,380000.0,380000,"jalan sri putri 7/x, taman putri, 81000, Johor",1.656731,103.572500,NaN,1540,SQUARE_FEET,Johor,Kulai,NaN,380000.0,342000.0,1732.863760
1,johor,sale,518000.0,518000,"jalan kebangsaan, Taman Universiti, Pulai Utama, 81300, Johor",1.533162,103.629529,309848,1300,SQUARE_FEET,Johor,Skudai,NaN,518000.0,466200.0,2362.166914
2,johor,sale,1380000.0,1380000,"jalan indah 14/x, Bukit Indah, 79100, Johor",1.483319,103.645116,3200,3367,SQUARE_FEET,Johor,Johor Bahru,NaN,1380000.0,1242000.0,6293.031548
3,johor,sale,313500.0,512260,"Bandar Putera Indah, Batu Pahat, Johor",1.938611,102.933500,26 acres,From 1498 - 1602,NaN,Johor,Batu Pahat,Emerald Garden 3,412880.0,371592.0,1882.802076
4,johor,sale,600000.0,600000,"Setia Tropika, Setia Tropika, 81200, Johor",NaN,NaN,1500,2100,SQUARE_FEET,Johor,Setia Tropika,NaN,600000.0,540000.0,2736.100673


In [71]:
# Income estimate
all_states['income'] = all_states['monthly_pmt']/0.35

In [72]:
# Creating income group based on income estimate
conditions = [
    (all_states['income'] <= 4849),
    (all_states['income'] >= 4850) & (all_states['income'] <= 10959),
    (all_states['income'] >= 10960) & (all_states['income'] <= 15039),
    (all_states['income'] > 15040)
    ]

# create a list of the values we want to assign for each condition
values = ['B40', 'M40','T20_T1','T20_T2']

# create a new column and use np.select to assign values to it using our lists as arguments
all_states['income_group'] = np.select(conditions, values)

# display updated DataFrame
all_states.head()

,state,type,min_price,max_price,address,lat,lng,landArea,builtUp,sizeUnit,level1,level2,level3,price_mean,principal_amt,monthly_pmt,income,income_group
0,johor,sale,380000.0,380000,"jalan sri putri 7/x, taman putri, 81000, Johor",1.656731,103.572500,NaN,1540,SQUARE_FEET,Johor,Kulai,NaN,380000.0,342000.0,1732.863760,4951.039313,M40
1,johor,sale,518000.0,518000,"jalan kebangsaan, Taman Universiti, Pulai Utama, 81300, Johor",1.533162,103.629529,309848,1300,SQUARE_FEET,Johor,Skudai,NaN,518000.0,466200.0,2362.166914,6749.048327,M40
2,johor,sale,1380000.0,1380000,"jalan indah 14/x, Bukit Indah, 79100, Johor",1.483319,103.645116,3200,3367,SQUARE_FEET,Johor,Johor Bahru,NaN,1380000.0,1242000.0,6293.031548,17980.090137,T20_T2
3,johor,sale,313500.0,512260,"Bandar Putera Indah, Batu Pahat, Johor",1.938611,102.933500,26 acres,From 1498 - 1602,NaN,Johor,Batu Pahat,Emerald Garden 3,412880.0,371592.0,1882.802076,5379.434504,M40
4,johor,sale,600000.0,600000,"Setia Tropika, Setia Tropika, 81200, Johor",NaN,NaN,1500,2100,SQUARE_FEET,Johor,Setia Tropika,NaN,600000.0,540000.0,2736.100673,7817.430494,M40


In [73]:
# all_states.to_csv('/Users/maryamnuri/Work/PROPERTY DATA/IPROPERTY/residential_sales/DATA/IPROPERTY_SALES_ALL_STATES.csv')

In [74]:
# all_states = pd.read_csv('/Users/maryamnuri/Work/PROPERTY DATA/IPROPERTY/residential_sales/DATA/iproperty_sales_all_states.csv')

In [21]:
# all_states = pd.read_csv('/Users/maryamnuri/Work/PROPERTY DATA/IPROPERTY/residential_sales/DATA/IPROPERTY_SALES_ALL_STATES.csv')

In [22]:
# Encode lat lon to geohash
import pygeohash as gh
all_states['GEOHASH6'] = all_states.apply(lambda x: gh.encode(x.lat, x.lng, precision=6), axis=1)

In [23]:
all_states.head()

,state,type,min_price,max_price,address,lat,lng,landArea,builtUp,sizeUnit,level1,level2,level3,price_mean,principal_amt,monthly_pmt,income,income_group,GEOHASH6
0,johor,sale,380000.0,380000,"jalan sri putri 7/x, taman putri, 81000, Johor",1.656731,103.572500,NaN,1540,SQUARE_FEET,Johor,Kulai,NaN,380000.0,342000.0,1732.863760,4951.039313,M40,w239kv
1,johor,sale,518000.0,518000,"jalan kebangsaan, Taman Universiti, Pulai Utama, 81300, Johor",1.533162,103.629529,309848,1300,SQUARE_FEET,Johor,Skudai,NaN,518000.0,466200.0,2362.166914,6749.048327,M40,w238wp
2,johor,sale,1380000.0,1380000,"jalan indah 14/x, Bukit Indah, 79100, Johor",1.483319,103.645116,3200,3367,SQUARE_FEET,Johor,Johor Bahru,NaN,1380000.0,1242000.0,6293.031548,17980.090137,T20_T2,w238qw
3,johor,sale,313500.0,512260,"Bandar Putera Indah, Batu Pahat, Johor",1.938611,102.933500,26 acres,From 1498 - 1602,NaN,Johor,Batu Pahat,Emerald Garden 3,412880.0,371592.0,1882.802076,5379.434504,M40,w235n2
4,johor,sale,600000.0,600000,"Setia Tropika, Setia Tropika, 81200, Johor",NaN,NaN,1500,2100,SQUARE_FEET,Johor,Setia Tropika,NaN,600000.0,540000.0,2736.100673,7817.430494,M40,000000


#### Getting the income group

In [24]:
## Read sample address data
df_address

/Users/maryamnuri/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (8,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
df_address.reset_index(drop=True, inplace=True)

In [1]:
df_address.columns

In [ ]:
## Convert lat long to geohash

In [31]:
df_address['GEOHASH6'] = df_address['GEOHASH'].str[:6]

In [3]:
df_address.head()

In [33]:
mapped_df = pd.merge(df_address,all_states,on='GEOHASH6',how='inner')

In [2]:
mapped_df.tail()

In [5]:
mapped_df.shape

In [4]:
mapped_df2 = mapped_df[['user','income_group']]
mapped_df2.head()

In [6]:
mapped_df2.user.nunique()